设计思路：

假设从某个时间点开始，每周淘汰最后10%。


In [37]:
import pandas as pd
import math
import gc
from IPython.display import display, HTML

import base
import fund_scanner.common_tools.database as db

engine = db.get_sqlalchemy_engine()

In [29]:
# 准备所有参赛者

sql = 'select * from funds a left join funds_update b on a.funds_id=b.funds_id'

df_all_candidates = pd.read_sql(sql, engine).iloc[:,[0,1,2,3,4,5,11]]

df_all_candidates = df_all_candidates.set_index('funds_id')

df_all_candidates['last_price'] = math.nan
df_all_candidates['current_price'] = math.nan
df_all_candidates['gain_ratio'] = math.nan
df_all_candidates['good'] = 0
good = {
    'not_in_yet' : 0,
    'healthy' : 1,
    'hurt_once' : 2,
    'hurt_twice' : 3,
    'out' : 10,
    'noway' : 11
    }
# good值:
#     0 : 未参赛
#     1 : 已参赛，正活跃
#     2 : 受伤 1次
#     3 : 受伤 2次
#     10 : 淘汰
#     11 : 没有参赛资格
# last_price: 上一轮净值
# current_price: 这一轮净值
# gain_ratio: 这一轮涨幅
df_all_candidates['score'] = 0

df_all_candidates = df_all_candidates.sort_values('funds_start_date')

df_all_candidates.tail(3)

,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,score
funds_id,,,,,,,,,,,
5418,001817,易方达瑞兴灵活配置混合I,None,NaT,2017-07-29 09:34:16,NaN,NaN,NaN,NaN,0,0
5426,001988,南方纯元A,None,NaT,2017-07-29 09:35:12,NaN,NaN,NaN,NaN,0,0
5439,004911,中加纯债定开债券A,None,NaT,2017-07-29 09:36:17,NaN,NaN,NaN,NaN,0,0


In [48]:
# game1 游戏规则
# 选择 2013年 前成立的老基金们参赛，从 2013年 开始，每 14天 比一次净值的涨幅，
# 把涨幅最靠后 8支 的淘汰，如果剩下基金比较少了，则淘汰排名最后的 1/10
# 一直跑到 today，
# 如果中途淘汰数量过多，最后剩下只有 5支 基金的时候，则直接中止比赛，
# 返回最后留下的基金
# 同时可以观测某一支基金是在哪个过程被淘汰的

def game1(funds_start_date='2013-1-1', competition_start_date='2013-1-1', competition_end_date='today',
          competition_time_span=14, bottom_to_be_out_max_count=8, bottom_to_be_out_percentage=10, 
          min_result_count=5, watching_code=None):
    
    gc.collect()
    # 重新读取数据
    df_competition = df_all_candidates.copy()

    # 2017年1月1日以后成立的基金或者没有写成立时间的基金没有参赛资格
    df_competition.loc[df_competition['funds_start_date']>pd.to_datetime(funds_start_date), 'good'] = good['out']
    df_competition.loc[df_competition['funds_start_date'].isnull(),'good'] = good['out']

    #资产规模小于1亿，或者没有数据的没有资格参赛
    df_competition.loc[df_competition['funds_amount']<1, 'good'] = good['out']
    df_competition.loc[df_competition['funds_amount'].isnull(),'good'] = good['out']


    #比赛从最早一直基金成立开始
    #start_point = pd.to_datetime(df_competition.iloc[0,3])
    #比赛从2013年1月1日开始
    start_point = pd.to_datetime(competition_start_date)
    round_count = 1
    current_date = start_point
    df_competition_results = []
    while current_date < pd.to_datetime(competition_end_date):
        # 观测特定基金的状态
        if watching_code is not None:
            display(df_competition.loc[df_competition['funds_code']==watching_code, :])
            
        # 得到当天价格（如果当天没几个价格就查次日的）
        df_competition['last_price'] = df_competition['current_price']
        while True:
            df_current_price = pd.read_sql('select * from funds_historical_price where funds_price_date=\'%s\''%current_date, engine)
            if len(df_current_price)>10:
                break
            else:
                current_date = current_date + pd.DateOffset(1)

        df_current_price = df_current_price.set_index('funds_id')
        df_competition['current_price'].update(df_current_price['funds_price_adjust'])

        # 如果价格从0到有，则说明参赛了，设置为 参赛1
        df_competition.loc[(df_competition['last_price']>0) & (df_competition['good']==good['not_in_yet']), 'good']=good['healthy']


        # 如果价格从非0到有，则说明价格变化了，计算变化率
        df_competition['gain_ratio'] = \
        ( df_competition['current_price'] - df_competition['last_price'] ) / df_competition['last_price']

        # 按变化率排序，将最后10%选手设置为 淘汰
        active_candidates = len(df_competition.loc[(df_competition['good']==good['healthy'])])
        print('Round %d: %s'% (round_count, current_date))
        if active_candidates>min_result_count:
            should_be_removed = active_candidates // bottom_to_be_out_percentage
            if should_be_removed > bottom_to_be_out_max_count:
                should_be_removed = bottom_to_be_out_max_count
            if should_be_removed>=1:
                current_competition = df_competition.loc[(df_competition['good']==good['healthy'])].sort_values('gain_ratio')
                min_gain_ratio = current_competition['gain_ratio'].min()
                max_gain_ratio = current_competition['gain_ratio'].max()
                threshold = current_competition.iloc[should_be_removed, :]['gain_ratio']
                print('最低涨幅 %f, 最高涨幅 %f ----> 淘汰 %d/%d 名选手，他们当期收益率低于 %f'\
                      %(min_gain_ratio, max_gain_ratio, should_be_removed, active_candidates, threshold))
                df_competition.loc[(df_competition['good']==good['healthy']) & (df_competition['gain_ratio']<=threshold), 'good'] = good['out']
            else:
                break

        round_count += 1
        current_date = current_date + pd.DateOffset(competition_time_span)
        df_competition_results.append(df_competition.copy())


    # The winner is:
    return df_competition.loc[df_competition['good']==good['healthy']].sort_values('current_price', ascending=False)

In [55]:
game1(competition_start_date='2015-05-23', competition_time_span=5, bottom_to_be_out_max_count=10, watching_code='540006')

,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,score
funds_id,,,,,,,,,,,
4269,540006,汇丰晋信大盘股票A,股票型,2009-06-24,2017-07-29 00:50:12,68.75,NaN,NaN,NaN,0,0


Round 1: 2015-05-25 00:00:00


,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,score
funds_id,,,,,,,,,,,
4269,540006,汇丰晋信大盘股票A,股票型,2009-06-24,2017-07-29 00:50:12,68.75,NaN,2.4858,NaN,0,0


Round 2: 2015-06-15 00:00:00
最低涨幅 -0.067426, 最高涨幅 0.209111 ----> 淘汰 30/823 名选手，他们当期收益率低于 -0.018140


,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,score
funds_id,,,,,,,,,,,
4269,540006,汇丰晋信大盘股票A,股票型,2009-06-24,2017-07-29 00:50:12,68.75,2.4858,2.5875,0.040912,1,0


...

,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,score
funds_id,,,,,,,,,,,
4269,540006,汇丰晋信大盘股票A,股票型,2009-06-24,2017-07-29 00:50:12,68.75,2.7889,2.9268,0.049446,10,0


Round 37: 2017-06-20 00:00:00
最低涨幅 0.001214, 最高涨幅 0.027292 ----> 淘汰 4/44 名选手，他们当期收益率低于 0.005924


,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,score
funds_id,,,,,,,,,,,
4269,540006,汇丰晋信大盘股票A,股票型,2009-06-24,2017-07-29 00:50:12,68.75,2.9268,2.9795,0.018006,10,0


Round 38: 2017-07-10 00:00:00
最低涨幅 0.001067, 最高涨幅 0.022782 ----> 淘汰 3/39 名选手，他们当期收益率低于 0.001657


,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,score
funds_id,,,,,,,,,,,
3093,163402,兴全趋势投资混合(LOF),混合型,2005-11-03,2017-07-30 10:02:04,83.56,8.8498,8.9489,0.011198,1,0
4501,288002,华夏收入混合,混合型,2005-11-17,2017-07-29 01:13:15,32.65,6.1860,6.2960,0.017782,1,0
3742,160505,博时主题行业混合(LOF),混合型,2005-01-06,2017-07-28 23:42:16,80.14,5.2610,5.3130,0.009884,1,0
3252,161005,富国天惠成长混合A,混合型,2005-11-16,2017-07-28 18:54:07,38.87,4.5231,4.5573,0.007561,1,0
4176,481001,工银核心价值混合A,混合型,2005-08-31,2017-07-29 00:41:07,44.03,4.4048,4.4404,0.008082,1,0
2933,100020,富国天益价值混合,混合型,2004-06-15,2017-07-30 07:20:04,30.89,4.3863,4.4391,0.012037,1,0
4056,151001,银河稳健,混合型,2003-08-04,2017-07-29 00:29:07,10.90,4.3699,4.4216,0.011831,1,0
4050,180010,银华优质增长混合,混合型,2006-06-09,2017-07-29 00:26:13,27.59,3.8851,3.9038,0.004813,1,0
4014,002031,华夏策略精选,混合型,2008-10-23,2017-07-29 00:23:04,10.39,3.8040,3.8700,0.017350,1,0
